# Golang Garble String Decryption
> Garble GO obfuscation string decryption

- toc: true 
- badges: true
- categories: [garble,go,obfuscation,strings]

## Overview

[Gable](https://github.com/burrowers/garble) is an open source obfuscation framework for GO which also includes a string encryption feature. It has been used in multiple GO based malware builds including [RootTeam](https://research.openanalysis.net/rootteam/stealer/triage/2023/07/20/rootteam.html) and [Bandit Stealer](https://research.openanalysis.net/bandit/stealer/garble/go/obfuscation/2023/07/31/bandit-garble.html). 

### String Obfuscation/Encryption
There is a weakness in the string encryption implementation that may be exploited to generically decrypt strings and identify Garble GO builds. Encrypted strings are placed in functions rather than in-lined. These functions follow a similar pattern that can be sigged.


## Sample
We will be using a Bandit Steal sample that has been obfuscated with Garble as our example `623a5f4c57cf5b3feb6775508cd6492f89d55ce11f62e0b6fb1020fd730b2e8f`.


## Analysis

### String Decryption Emulation

String retrieval is a simple matter of emulating the string decryption function.

In [79]:
from unicorn import *
from unicorn.x86_const import *
import struct
from capstone import *
from capstone.x86 import *


def decrypt(code):
    uc = Uc(UC_ARCH_X86, UC_MODE_64)

    # Setup the stack
    stack_base = 0x00100000
    stack_size = 0x00100000
    RSP = stack_base + (stack_size // 2)
    uc.mem_map(stack_base, stack_size)
    uc.mem_write(stack_base, b"\x00" * stack_size)

    uc.reg_write(UC_X86_REG_RSP, RSP)

    # Setup code 
    target_base = 0x00400000
    target_size = 0x00100000
    target_end = target_base + len(code)

    uc.mem_map(target_base, target_size, UC_PROT_ALL)
    uc.mem_write(target_base, b"\x00" * target_size)
    uc.mem_write(target_base, code)


    data_base = 0x00600000
    data_size = 0x00100000

    uc.mem_map(data_base, data_size, UC_PROT_ALL)
    uc.mem_write(data_base, b"\x00" * data_size)


    cs = Cs(CS_ARCH_X86, CS_MODE_64)
    cs.detail = True

    def trace(uc, address, size, user_data):
        insn = next(cs.disasm(uc.mem_read(address, size), address))
        #print(f"{address:#010x}:\t{insn.mnemonic}\t{insn.op_str}")
        if insn.mnemonic == 'call':
            #print("Ending on a call!")
            uc.emu_stop()

    uc.reg_write(UC_X86_REG_R14, data_base)
    uc.hook_add(UC_HOOK_CODE, trace, None)
    uc.emu_start(target_base, target_end, 0, 0)

    #print(uc.mem_read(stack_base, stack_size).replace(b'\x00', b''))
    ptr_string = uc.reg_read(UC_X86_REG_RBX)
    size = uc.reg_read(UC_X86_REG_RCX)
    string_data = uc.mem_read(ptr_string, size)
    string = string_data.decode('utf-8')
    return string


memory_fails =[]
def decrypt_mem_hack(code):
    global memory_fails
    uc = Uc(UC_ARCH_X86, UC_MODE_64)

    # Setup the stack
    stack_base = 0x00100000
    stack_size = 0x00100000
    RSP = stack_base + (stack_size // 2)
    uc.mem_map(stack_base, stack_size)
    uc.mem_write(stack_base, b"\x00" * stack_size)

    uc.reg_write(UC_X86_REG_RSP, RSP)

    # Setup code 
    target_base = 0x00400000
    target_size = 0x00100000
    target_end = target_base + len(code)

    uc.mem_map(target_base, target_size, UC_PROT_ALL)
    uc.mem_write(target_base, b"\x00" * target_size)
    uc.mem_write(target_base, code)


    data_base = 0x00600000
    data_size = 0x00100000

    uc.mem_map(data_base, data_size, UC_PROT_ALL)
    uc.mem_write(data_base, b"\x00" * data_size)


    cs = Cs(CS_ARCH_X86, CS_MODE_64)
    cs.detail = True

    def trace(uc, address, size, user_data):
        insn = next(cs.disasm(uc.mem_read(address, size), address))
        #print(f"{address:#010x}:\t{insn.mnemonic}\t{insn.op_str}")
        if insn.mnemonic == 'call':
            #print("Ending on a call!")
            uc.emu_stop()
            
    def hook_mem_invalid(uc, access, address, size, value, user_data):
        # print("handling memory error")
        # rip = uc.reg_read(UC_X86_REG_RIP)
        # tmp_code = uc.mem_read(rip, 15)
        # insn = next(cs.disasm(tmp_code, 0))
        # print(f"\tRIP: {hex(rip)}")
        # print(f"\tRead address: {hex(address)}")
        # print(f"\tRead size: {size}")
        # print(f"\tInstruction size: {insn.size}")
        # print(f"\t{insn.mnemonic}\t{insn.op_str}")
        #uc.mem_write(rip, b'\x90'*insn.size)
        memory_fails.append((address,size))
        return True
    
    uc.hook_add(UC_HOOK_MEM_INVALID , hook_mem_invalid)
    
    uc.reg_write(UC_X86_REG_R14, data_base)
    uc.hook_add(UC_HOOK_CODE, trace, None)
    uc.emu_start(target_base, target_end, 0, 0)

    #print(uc.mem_read(stack_base, stack_size).replace(b'\x00', b''))
    ptr_string = uc.reg_read(UC_X86_REG_RBX)
    size = uc.reg_read(UC_X86_REG_RCX)
    string_data = uc.mem_read(ptr_string, size)
    string = string_data.decode('utf-8')
    return string


def decrypt_fix_mem(code, memory_chunks):
    uc = Uc(UC_ARCH_X86, UC_MODE_64)

    # Setup the stack
    stack_base = 0x00100000
    stack_size = 0x00100000
    RSP = stack_base + (stack_size // 2)
    uc.mem_map(stack_base, stack_size)
    uc.mem_write(stack_base, b"\x00" * stack_size)

    uc.reg_write(UC_X86_REG_RSP, RSP)

    # Setup code 
    target_base = 0x00400000
    target_size = 0x00100000
    target_end = target_base + len(code)

    uc.mem_map(target_base, target_size, UC_PROT_ALL)
    uc.mem_write(target_base, b"\x00" * target_size)
    uc.mem_write(target_base, code)


    data_base = 0x00600000
    data_size = 0x00100000

    uc.mem_map(data_base, data_size, UC_PROT_ALL)
    uc.mem_write(data_base, b"\x00" * data_size)
    
    # setup memory hacks
    for mem in memory_chunks:
        #print(f"adding memory at {hex(mem[0])} size {hex(mem[1])}")
        if mem[0] == 0:
            continue
        kb = 4*1024
        addr = (mem[0]//kb)*kb 
        addr -= kb
        #print(f"corrected memory at {hex(addr)} size {hex(kb*2)}")
        try:
            uc.mem_map(addr, kb*2, UC_PROT_ALL)
            uc.mem_write(addr, b"\x00" * kb*2)
        except:
            pass


    cs = Cs(CS_ARCH_X86, CS_MODE_64)
    cs.detail = True

    def trace(uc, address, size, user_data):
        insn = next(cs.disasm(uc.mem_read(address, size), address))
        #print(f"{address:#010x}:\t{insn.mnemonic}\t{insn.op_str}")
        if insn.mnemonic == 'call':
            #print("Ending on a call!")
            uc.emu_stop()

    uc.reg_write(UC_X86_REG_R14, data_base)
    uc.hook_add(UC_HOOK_CODE, trace, None)
    uc.emu_start(target_base, target_end, 0, 0)

    #print(uc.mem_read(stack_base, stack_size).replace(b'\x00', b''))
    ptr_string = uc.reg_read(UC_X86_REG_RBX)
    size = uc.reg_read(UC_X86_REG_RCX)
    string_data = uc.mem_read(ptr_string, size)
    string = string_data.decode('utf-8')
    return string

#print(decrypt('49 3B 66 10 0F 86 C8 00 00 00 48 83 EC 38 48 89 6C 24 30 48 8D 6C 24 30 48 BA E5 B1 F0 56 65 EA 73 C9 48 89 54 24 18 66 C7 44 24 20 6F 6E 48 BA 02 00 01 05 01 05 05 07 48 89 54 24 22 48 BA 05 07 05 02 00 07 07 05 48 89 54 24 28 31 C0 EB 1D 44 0F B6 4C 34 18 41 29 D1 41 8D 51 E1 88 54 3C 18 41 8D 50 E1 88 54 34 18 48 83 C0 02 48 83 F8 0E 7D 27 0F B6 54 04 22 0F B6 74 04 23 89 D7 31 F2 01 C2 48 83 FF 0A 73 3C 44 0F B6 44 3C 18 41 29 D0 48 83 FE 0A 72 B8 EB 1B 31 C0 48 8D 5C 24 18 B9 0A 00 00 00 E8 D5 B8 88 FF 48 8B 6C 24 30 48 83 C4 38 C3 89 F0 B9 0A 00 00 00 0F 1F 40 00 E8 3B 02 8A FF 89 F8 B9 0A 00 00 00 E8 2F 02 8A FF 90 E8 A9 DB 89 FF E9 24 FF FF FF CC CC CC CC'))
#print(decrypt('4C 8D 64 24 C8 4D 3B 66 10 0F 86 57 01 00 00 48 81 EC B8 00 00 00 48 89 AC 24 B0 00 00 00 48 8D AC 24 B0 00 00 00 48 BA 03 DC F5 44 2F 20 21 53 48 89 54 24 1D 48 BA AA 6D 47 01 6F 45 3A 04 48 89 54 24 25 48 BA 01 6F 45 3A 04 01 23 95 48 89 54 24 28 48 BA 72 19 F5 3B 01 EA 20 47 48 89 54 24 30 48 BA A6 18 DF 72 1B 69 53 4E 48 89 54 24 38 48 BA 4D 76 17 0A 06 E4 23 57 48 89 54 24 40 48 BA 09 85 2F E6 28 FC 36 02 48 89 54 24 48 48 BA B2 49 E8 14 19 4D 29 64 48 89 54 24 50 48 BA 2C 2F 26 12 27 1B 32 0B 48 89 54 24 58 48 8D 7C 24 60 48 8D 35 E7 00 A6 00 0F 1F 80 00 00 00 00 48 89 6C 24 F0 48 8D 6C 24 F0 E8 AB D9 FE FF 48 8B 6D 00 31 C0 EB 1A 44 0F B6 4C 34 1D 41 83 C1 E0 44 01 CA 88 54 3C 1D 44 88 44 34 1D 48 83 C0 02 48 83 F8 58 7D 31 0F B6 54 04 58 0F B6 74 04 59 89 D7 31 F2 01 C2 48 83 FF 3B 73 48 44 0F B6 44 3C 1D 41 83 C0 E0 41 01 D0 66 0F 1F 44 00 00 48 83 FE 3B 72 B1 EB 21 31 C0 48 8D 5C 24 1D B9 3B 00 00 00 E8 47 89 FD FF 48 8B AC 24 B0 00 00 00 48 81 C4 B8 00 00 00 C3 89 F0 B9 3B 00 00 00 E8 AB D2 FE FF 89 F8 B9 3B 00 00 00 0F 1F 40 00 E8 9B D2 FE FF 90 E8 15 AC FE FF E9 90 FE FF FF CC CC CC CC CC CC CC CC CC CC CC CC CC CC CC CC'))
#print(decrypt('49 3B 66 10 0F 86 C8 00 00 00 48 83 EC 38 48 89 6C 24 30 48 8D 6C 24 30 48 BA E5 B1 F0 56 65 EA 73 C9 48 89 54 24 18 66 C7 44 24 20 6F 6E 48 BA 02 00 01 05 01 05 05 07 48 89 54 24 22 48 BA 05 07 05 02 00 07 07 05 48 89 54 24 28 31 C0 EB 1D 44 0F B6 4C 34 18 41 29 D1 41 8D 51 E1 88 54 3C 18 41 8D 50 E1 88 54 34 18 48 83 C0 02 48 83 F8 0E 7D 27 0F B6 54 04 22 0F B6 74 04 23 89 D7 31 F2 01 C2 48 83 FF 0A 73 3C 44 0F B6 44 3C 18 41 29 D0 48 83 FE 0A 72 B8 EB 1B 31 C0 48 8D 5C 24 18 B9 0A 00 00 00 E8 D5 B8 88 FF 48 8B 6C 24 30 48 83 C4 38 C3 89 F0 B9 0A 00 00 00 0F 1F 40 00 E8 3B 02 8A FF 89 F8 B9 0A 00 00 00 E8 2F 02 8A FF 90 E8 A9 DB 89 FF E9 24 FF FF FF'))

### String Decryption Identification

It appears that the string decryption functions share the same epilogue (possibly because of templating?) they all pass the decrypted byte string to `slicebytetostring` to create a GO string, then return. We might be able to use this as a signature to identify and extract these functions.

#### Eplilogue
```
48 8D 5C 24 28                          lea     rbx, [rsp+58h+var_30]
B9 0D 00 00 00                          mov     ecx, 0Dh
E8 9A 30 F3 FF                          call    runtime_slicebytetostring
48 8B 6C 24 50                          mov     rbp, [rsp+58h+var_8]
48 83 C4 58                             add     rsp, 58h
C3                                      retn
```

```
48 8D 5C 24 18                          lea     rbx, [rsp+38h+var_20]
B9 0A 00 00 00                          mov     ecx, 0Ah
E8 D5 B8 88 FF                          call    runtime_slicebytetostring
48 8B 6C 24 30                          mov     rbp, [rsp+38h+var_8]
48 83 C4 38                             add     rsp, 38h
C3                                      retn
```

Common epilogue bytes...
```
48 8D 5C ?? ?? B9 ?? ?? ?? ?? E8 ?? ?? ?? ?? 48 8B 6C ?? ?? 48 83 ?? ?? C3 
```

#### Prologue

Many of the function have a stack size check that can be used to identify the start of the function... this may not be universal??

```
49 3B 66 10                             cmp     rsp, [r14+10h]
0F 86 E6 01 00 00                       jbe     loc_516A90
48 83 EC 58                             sub     rsp, 58h
```

```
49 3B 66 10                             cmp     rsp, [r14+10h]
0F 86 08 01 00 00                       jbe     loc_140136672
48 83 EC 60                             sub     rsp, 60h
```

Common prologue bytes...
```
49 3B 66 10 0F 86 
```



In [81]:
import re
import pefile

file_data = open('/tmp/bandit_obfuscated.bin', 'rb').read()
pe = pefile.PE(data=file_data)

section_data = None

for s in pe.sections:
    if b'.text\x00' == s.Name[:6]:
        section_data = s.get_data()
        
assert section_data is not None

egg = rb'\x48\x8D\x5C..\xB9....\xE8....\x48\x8B\x6C..\x48\x83..\xC3'

functions = []

for m in re.finditer(egg, section_data, re.DOTALL):
    end = m.end()
    tmp_data = section_data[:end]
    start = tmp_data.rfind(b'\x49\x3B\x66\x10\x0F\x86')
    if start == -1:
        continue
        
    tmp_data = tmp_data[start:]
    functions.append(tmp_data)
    
# for fn in functions:
#     try:
#         #s = decrypt(fn)
#         s = decrypt_mem_hack(fn)
#         #print(s)
#     except Exception as e:
#         pass
    
# for fn in functions:
#     try:
#         #s = decrypt(fn)
#         s = decrypt_fix_mem(fn, memory_fails)
#         print(s)
#     except Exception as e:
#         print(f"----------------------------------- Failed for {e}")
    

for fn in functions:
    try:
        s = decrypt(fn)
        print(s)
    except Exception as e:
        pass

complex128
interface
unsafe.Pointer
reflect: call of 
 on zero Value
reflect.Value.Len
reflect: NumIn of non-func type
reflect: In of non-func type
non-empty string
read varint
t.xcount > 0
sync: Unlock of unlocked RWMutex
sync.Cond is copied
short buffer
unexpected EOF
Anatolian_Hieroglyphs
Armenian
Balinese
Bopomofo
Canadian_Aboriginal
Caucasian_Albanian
Caucasian_Albanian
Cyrillic
Dives_Akuru
Georgian
Glagolitic
Gujarati
Gunjala_Gondi
Hiragana
Imperial_Aramaic
Javanese
Kayah_Li
Kharoshthi
Linear_B
Malayalam
Masaram_Gondi
Medefaidrin
Meetei_Mayek
Mende_Kikakui
Meroitic_Cursive
Mongolian
Mongolian
Nandinagari
New_Tai_Lue
Old_Hungarian
Old_Italic
Old_North_Arabian
Old_North_Arabian
Old_Persian
Old_Sogdian
Old_South_Arabian
Old_Turkic
Pahawh_Hmong
Pahawh_Hmong
Phoenician
Samaritan
Sora_Sompeng
Sora_Sompeng
Syloti_Nagri
Tai_Viet
Ugaritic
Zanabazar_Square
Bidi_Control
Deprecated
Hex_Digit
IDS_Binary_Operator
IDS_Trinary_Operator
Other_Default_Ignorable_Code_Point
Other_Lowercase
Pattern_S

#### Bugs

There is an issue with functions that read globals (or uninitialized memory in general) we tried to handle these with a hook on `UC_HOOK_MEM_INVALID` but there is some issue with Unicorn where the execution does not continue even if the exception is handled. Not sure why?? Unicorn bug??

In [78]:



code = bytes.fromhex('49 3B 66 10 0F 86 DB 00 00 00 48 83 EC 50 48 89 6C 24 48 48 8D 6C 24 48 0F 1F 84 00 00 00 00 00 48 39 05 09 92 BB 00 74 78 48 39 05 F8 91 BB 00 74 5F 90 48 BA B7 72 27 62 BE 45 06 84 48 89 54 24 33 48 BA 45 06 84 0C 39 63 7F D8 48 89 54 24 38 48 BA 8F 8D BB A9 85 E7 E5 26 48 89 54 24 40 48 BA 2C E0 92 D0 2D BC 74 A4 48 89 54 24 1E 48 BA BC 74 A4 71 A6 D3 F3 51 48 89 54 24 23 48 BA AF D0 2A 17 F9 4C 5D 9A 48 89 54 24 2B 31 C0 EB 33 E8 8A 3A 00 00 48 8B 6C 24 48 48 83 C4 50 90 C3 E8 9A 36 00 00 48 8B 6C 24 48 48 83 C4 50 C3 0F B6 54 04 33 0F B6 74 04 1E 29 D6 40 88 74 04 1E 48 FF C0 48 83 F8 15 7C E6 31 C0 48 8D 5C 24 1E B9 15 00 00 00 E8 05 02 E4 FF 48 8B 6C 24 48 48 83 C4 50 C3 48 89 44 24 08')

from unicorn import *
from unicorn.x86_const import *
import struct
from capstone import *
from capstone.x86 import *


def decrypt2(code):
    uc = Uc(UC_ARCH_X86, UC_MODE_64)

    # Setup the stack
    stack_base = 0x00100000
    stack_size = 0x00100000
    RSP = stack_base + (stack_size // 2)
    uc.mem_map(stack_base, stack_size)
    uc.mem_write(stack_base, b"\x00" * stack_size)

    uc.reg_write(UC_X86_REG_RSP, RSP)

    # Setup code 
    target_base = 0x00400000
    target_size = 0x00100000
    target_end = target_base + len(code)

    uc.mem_map(target_base, target_size, UC_PROT_ALL)
    uc.mem_write(target_base, b"\x00" * target_size)
    uc.mem_write(target_base, code)


    data_base = 0x00600000
    data_size = 0x00100000

    uc.mem_map(data_base, data_size, UC_PROT_ALL)
    uc.mem_write(data_base, b"\x00" * data_size)


    cs = Cs(CS_ARCH_X86, CS_MODE_64)
    cs.detail = True

    def trace(uc, address, size, user_data):
        insn = next(cs.disasm(uc.mem_read(address, size), address))
        print(f"{address:#010x}:\t{insn.mnemonic}\t{insn.op_str}")
        if insn.mnemonic == 'call':
            #print("Ending on a call!")
            uc.emu_stop()
            
    def hook_mem_invalid(uc, access, address, size, value, user_data):
        print("handling memory error")
        rip = uc.reg_read(UC_X86_REG_RIP)
        tmp_code = uc.mem_read(rip, 15)
        insn = next(cs.disasm(tmp_code, 0))
        print(f"\tRIP: {hex(rip)}")
        print(f"\tRead address: {hex(address)}")
        print(f"\tRead size: {size}")
        print(f"\tInstruction size: {insn.size}")
        print(f"\t{insn.mnemonic}\t{insn.op_str}")
        #uc.mem_write(rip, b'\x90'*insn.size)
        kb = 4*1024
        emu.mem_map((address//kb)*kb, kb)
        emu.mem_write(address, b'\x00'*kb)
        return True

#     def hook_mem_access(emu, access, address, size, value, user_data):
#         print(f"memory access: address {hex(address)} size:{size}")
#         kb = 4*1024
#         if not (target_base <= address <= target_base + target_size) and not (stack_base <= address <= stack_base + stack_size) and not (data_base <= address <= data_base + data_size):
#             print("\tCreating memory")
#             emu.mem_map((address/kb)*kb, kb)
#             emu.mem_write(address, b'\x00'*kb)

            
#     uc.hook_add(UC_HOOK_MEM_READ | UC_HOOK_MEM_WRITE , hook_mem_access)
    
    uc.hook_add(UC_HOOK_MEM_INVALID , hook_mem_invalid)
        
    uc.reg_write(UC_X86_REG_R14, data_base)
    uc.hook_add(UC_HOOK_CODE, trace, None)
    
    uc.emu_start(target_base, target_end, 0, 0)

    #print(uc.mem_read(stack_base, stack_size).replace(b'\x00', b''))
    ptr_string = uc.reg_read(UC_X86_REG_RBX)
    size = uc.reg_read(UC_X86_REG_RCX)
    string_data = uc.mem_read(ptr_string, size)
    string = string_data.decode('utf-8')
    return string

decrypt_fix_mem(code, [(0xfb9230,8)])
#decrypt2(code)


adding memory at 0xfb9230 size 0x8


''

In [53]:

hex(0xbb9209 + 0x400020)

'0xfb9229'

### Testing


In [82]:
import re
import pefile


def pull_strings(file_path):
    file_data = open(file_path, 'rb').read()
    pe = pefile.PE(data=file_data)

    section_data = None

    for s in pe.sections:
        if b'.text\x00' == s.Name[:6]:
            section_data = s.get_data()

    assert section_data is not None

    egg = rb'\x48\x8D\x5C..\xB9....\xE8....\x48\x8B\x6C..\x48\x83..\xC3'

    functions = []

    for m in re.finditer(egg, section_data, re.DOTALL):
        end = m.end()
        tmp_data = section_data[:end]
        start = tmp_data.rfind(b'\x49\x3B\x66\x10\x0F\x86')
        if start == -1:
            continue

        tmp_data = tmp_data[start:]
        functions.append(tmp_data)

    for fn in functions:
        try:
            s = decrypt(fn)
            print(f"\t{s}")
        except Exception as e:
            pass
        
        
import os
# assign directory
directory = '/tmp/test'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f"Testing {f}")
        try:
            pull_strings(f) 
        except:
            pass
        

Testing /tmp/test/cba236aff9c3379946527145a5604eaac6d4c691343b32c625386b3df06c9f4f
	reflect: slice index out of range
	j < s.Len
	6103515625
	30517578125
	762939453125
	476837158203125
	2384185791015625
	59604644775390625
	1490116119384765625
	7450580596923828125
	37252902984619140625
	186264514923095703125
	2910383045673370361328125
	227373675443232059478759765625
	3552713678800500929355621337890625
	444089209850062616169452667236328125
	strconv: illegal AppendInt/FormatInt base
	0123456789ABCDEF
	sync: negative WaitGroup counter
	Anatolian_Hieroglyphs
	Buginese
	Chorasmian
	Chorasmian
	Cyrillic
	Devanagari
	Ethiopic
	Glagolitic
	Gunjala_Gondi
	Gurmukhi
	Hiragana
	Imperial_Aramaic
	Kayah_Li
	Kharoshthi
	Linear_A
	Linear_B
	Medefaidrin
	Meetei_Mayek
	New_Tai_Lue
	Old_Italic
	Old_Turkic
	Samaritan
	Saurashtra
	Sora_Sompeng
	Ugaritic
	Diacritic
	IDS_Trinary_Operator
	Join_Control
	Other_Math
	Quotation_Mark
	Sentence_Terminal
	Variation_Selector
	complex128
	reflect.Copy
	Interface
	meth

## Testing Gobfuscate

We can also test with [gobfuscate](https://github.com/unixpickle/gobfuscate) another GO obfuscator that appears to use functions for encrypted strings. The following is a test with `bcddca962a619baaee36eb2e3dbf41eeb00f0e375888d2bba6a06584d2ba7edc`


In [83]:
pull_strings('/tmp/gob.bin')

	127.0.0.1
	Address
	%s%s%s
	main.exe
	146.70.104.249
	port for listen to
	isUseSystemProxy
	try to get system proxy
	certFingerprint
	readWriteTimeout
	sleepTimeout
	numberOfThreads
	thread launch delay(s)
	HTTPS_PROXY
	hostname_error 
	fqdn_error_new 
	outbound_ip: %s

	%s:%d

	Failed to connect T: 
	Empty response

	version.com
	otpt_error 
	kill.com
	Error handleDest: %s

